In [8]:
!pip install stable-baselines3

import torch
import gym
from stable_baselines3 import GAIL, SAC
from stable_baselines3.gail import ExpertDataset, generate_expert_traj

     |████████████████████████████████| 153kB 8.0MB/s 


ImportError: ignored

In [ ]:
!pip install gym
!pip install PyBullet
!apt-get update
!apt -q install imagemagick
!apt install -q xvfb
!pip install -q pyvirtualdisplay

     |████████████████████████████████| 89.3MB 60kB/s 
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease

In [ ]:
# imports
import numpy as np
import pandas as pd
import random
import sys
import time
from builtins import super
from collections import namedtuple
from __future__ import print_function
from pathlib import Path
from tqdm.auto import tqdm

# imports
from copy import deepcopy
from typing import NamedTuple
import time




# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable as V
from torch.distributions import Normal
import scipy.signal
from torch.optim import Adam

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu") # CPU actually trains faster because there is no batch

# IPython
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython import display
from IPython.display import Image # display
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Gym
import gym
import gym.spaces
gym.logger.set_level(gym.logger.ERROR)

# Gym rendering
from pyvirtualdisplay import Display

d = Display()
d.start();

#@markdown ## Figure setting
import matplotlib.pyplot as plt
from matplotlib import animation

%matplotlib inline 
%config InlineBackend.figure_format = 'retina'

plt.rcParams.update(plt.rcParamsDefault)
plt.rc('animation', html='jshtml')

In [ ]:
#@markdown ## Helper functions
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def soft_update_from_to(source, target, tau):
  for target_param, param in zip(target.parameters(), source.parameters()):
    target_param.data.copy_(
      target_param.data * (1.0 - tau) + param.data * tau
  )


class Batch(NamedTuple):
  state: torch.Tensor
  action: torch.Tensor
  reward: torch.Tensor
  discount: torch.Tensor
  next_state: torch.Tensor


class ReplayBuffer:
  def __init__(self, state_dim, action_dim, buffer_size):
    self.ptr = 0
    self.n_samples = 0
    self.buffer_size = buffer_size
    
    self.state = np.zeros((buffer_size, state_dim), dtype=np.float32)        
    self.action = np.zeros((buffer_size, action_dim), dtype=np.float32)
    self.reward = np.zeros(buffer_size, dtype=np.float32)
    self.discount = np.zeros(buffer_size, dtype=np.float32)
    self.next_state = np.zeros((buffer_size, state_dim), dtype=np.float32)

  def add(self, state, action, reward, discount, next_state):
    self.state[self.ptr] = state
    self.action[self.ptr] = action
    self.reward[self.ptr] = reward
    self.discount[self.ptr] = discount
    self.next_state[self.ptr] = next_state

    if self.n_samples < self.buffer_size:
      self.n_samples += 1

    self.ptr = (self.ptr + 1) % self.buffer_size

  def sample(self, batch_size):
    # Select batch_size number of sample indicies at random from the buffer
    idx = np.random.choice(self.n_samples, batch_size)    
    # Using the random indices, assign the corresponding state, action, reward,
    # discount, and next state samples.
    state = self.state[idx]
    action = self.action[idx]
    reward = self.reward[idx]
    discount = self.discount[idx]
    next_state = self.next_state[idx]
      
    return Batch(state=torch.as_tensor(state, dtype=torch.float32).to(device),
                 action=torch.as_tensor(action, dtype=torch.float32).to(device),
                 reward=torch.as_tensor(reward, dtype=torch.float32).to(device),
                 discount=torch.as_tensor(discount, dtype=torch.float32).to(device),
                 next_state=torch.as_tensor(next_state, dtype=torch.float32).to(device))


def learn_env(env, agent, gamma, n_steps):
  ep_reward = []
  ep_steps = []
  total_reward = 0
  t = 0
  tic = time.time()
  state = env.reset()
  for step in range(n_steps):
    action = agent.act(state)
    next_state, reward, done, _ = env.step(action)    
    discount = gamma*(1-done)
    
    total_reward += reward  
  
    agent.train(state, action, reward, discount, next_state, step)

    if done:
      toc = time.time()      
      print(f"Episode: {len(ep_reward) + 1}, reward: {total_reward:0.2f}, time:{toc-tic:0.2f}")
      tic = toc

      state = env.reset()      
      ep_reward.append(total_reward)
      total_reward = 0
      ep_steps.append(t)
      t = 0
    else:
      state = next_state
      t += 1

  return ep_reward

def visualize(env, agent, ep_steps):
    # Simulate agent in environment
    frames = []
    state = env.reset()
    for s in range(ep_steps):
        action = agent.act(state)
        next_state, _, done, _ = env.step(action)

        display.clear_output(wait=True)
        frames.append(env.render('rgb_array'))
        state = next_state
        if done:
            break
    
    # Generate video
    plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi=72)
    patch = plt.imshow(frames[0])
    plt.axis('off')
    animate = lambda i: patch.set_data(frames[i])
    anim = animation.FuncAnimation(plt.gcf(), animate, frames=range(len(frames)), interval=50)
    plt.close() # avoid showing extra plots
    return anim

In [ ]:
import gym
import pybullet_envs

In [ ]:
class Actor(nn.Module):
  def __init__(self, state_dim, action_dim, action_limit):
    super().__init__()    
    self.fc1 = nn.Linear(state_dim, 256)
    self.fc2 = nn.Linear(256, 256)
    self.fc3 = nn.Linear(256, action_dim)    
    self.action_limit = action_limit

  def forward(self, state):
    # Pass the state through the first layer and apply a ReLU activation
    x = F.relu(self.fc1(state))
    # Next pass the result through the second layer and apply a ReLU activation
    x = F.relu(self.fc2(x))
    # Now pass the result through the third layer and apply a tanh activation
    x = torch.tanh(self.fc3(x))
    # Finally scale the result by the action limit and return
    x = torch.multiply(x, self.action_limit) #x /= self.action_limit
    return x

class Critic(nn.Module):
  def __init__(self, state_dim, action_dim):
    super().__init__()
    self.fc1 = nn.Linear(state_dim + action_dim, 256)
    self.fc2 = nn.Linear(256, 256)
    self.fc3 = nn.Linear(256, 1)

  def forward(self, state, action):
    # First concatenate the state and action together
    x = torch.cat((state, action),1)
    # Next pass the result through the first layer and apply a ReLU activation
    x = F.relu(self.fc1(x))
    # Then pass the result through the second layer and apply a ReLU activation
    x = F.relu(self.fc2(x))
    # Finally  pass the result through the third layer
    x = self.fc3(x)
    # Ensure result has the right dimensions and return
    x = torch.squeeze(x, -1)
    return x  

class DDPGAgent:
  def __init__(self, actor, actor_optimizer, critic, critic_optimizer, tau,
               replay_buffer, batch_size, train_start, update_every, action_space, epsilon):
    self.actor = actor
    # Copy the actor network to get the target and disable autograd for performance
    self.actor_target = deepcopy(actor)
    for p in self.actor_target.parameters():
      p.requires_grad = False
    self.actor_optimizer = actor_optimizer

    self.critic = critic
    # Copy the critic network to get the target and disable autograd for performance
    self.critic_target = deepcopy(critic)
    for p in self.critic_target.parameters():
      p.requires_grad = False
    self.critic_optimizer = critic_optimizer

    self.tau = tau

    self.replay_buffer = replay_buffer
    self.batch_size = batch_size
    self.train_start = train_start
    self.update_every = update_every
    self.epsilon = epsilon
    self.action_space = action_space
    self.is_waiting = True
  
  def act(self, state):
    # Disable autograd for performance when choosing an action
    with torch.no_grad():
      # if we are in pre-training, sample randomly from the action space
      if self.is_waiting:
        return self.action_space.sample()
  
      # get an action from our policy i.e. actor network
      action = self.actor(torch.as_tensor(state, dtype=torch.float32).to(device))
      # apply standard normal noise to the action value to encourage exploration
      action = action.cpu().numpy() + self.epsilon * np.random.standard_normal(action.shape)
      return action
        
  def train(self, state, action, reward, discount, next_state, t):
    self.replay_buffer.add(state, action, reward, discount, next_state)  
    
    if t < self.train_start:
      return
    
    if t == self.train_start:
      self.is_waiting = False

    # batch update every several steps instead of once per step
    if t % self.update_every == 0:
      for _ in range(self.update_every):
        batch = self.replay_buffer.sample(self.batch_size)
        self.update(batch)
  
  def update(self, batch):
    # First, let's train the Critic network
    
    # Compute the predicted q-value from the critic network with the state and
    # action
    q_pred = self.critic(batch.state, batch.action)
    
    # Now compute the q-value target (also called td target or bellman backup)
    # we don't need to compute gradients on the q-value target, just the q-value
    # prediction, so we disable autograd here to speed up performance
    with torch.no_grad():
      # First, we will use the actor_target network to predict the best action
      # for the next state
      actor_target_action = self.actor_target(batch.state)
      # Next we will get the q-value target from our critic_target network,
      # using the the next state and the action we just got
      q_target = self.critic_target(batch.next_state,actor_target_action)
      # Next apply the reward and discount to get the q-value target
      q_target = batch.reward + batch.discount*q_target

    # Compute the MSE loss between the predicted and target values
    loss_critic = F.mse_loss(q_pred,q_target)

    # backpropogation to update the critic network
    self.critic_optimizer.zero_grad()    
    loss_critic.backward()
    self.critic_optimizer.step()

    # Next, let's train the Actor network

    # We disable autograd for the critic network during this phase to increase
    # performance
    for p in self.critic.parameters():
      p.requires_grad = False

    # Get the predicted action for the state from the actor network
    actor_action = self.actor(batch.next_state)
    # Use this action along with the current state to see what q-value the
    # critic network produces
    q_actor = self.critic(batch.next_state,actor_action)
    # Since we want to maximize the value, our loss is just the negative of the
    # mean of the batch of q-values we just got
    loss_actor = -torch.mean(q_actor)

    # backpropogation to update the critic network
    self.actor_optimizer.zero_grad()    
    loss_actor.backward()
    self.actor_optimizer.step()

    # Important! We need to unfreeze our critic network so it can be trained on
    # the next step
    for p in self.critic.parameters():
      p.requires_grad = True

    # Finally, soft update the target networks for both the actor and critic
    with torch.no_grad():
      soft_update_from_to(self.actor, self.actor_target, self.tau)
      soft_update_from_to(self.critic, self.critic_target, self.tau)

In [ ]:
# hyperparameters
seed = 522
buffer_size = 10000
gamma = 0.99
tau = 0.005
epsilon = 0.1
batch_size = 100
train_start = 1000
update_every = 50
n_steps = 30000

# environment setup
torch.manual_seed(seed)
np.random.seed(seed)

env = gym.make("HumanoidBulletEnv-v0") #AntBulletEnv-v0")
 #env = gym.make('Pendulum-v0')
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
action_limit = env.action_space.high[0]

# Create actor-critic modules and optimizers
actor = Actor(state_dim, action_dim, action_limit).to(device)
critic = Critic(state_dim, action_dim).to(device)
actor_optimizer = Adam(actor.parameters(), lr=1e-3)
critic_optimizer = Adam(critic.parameters(), lr=1e-3)

# Initialize our experience replay buffer
replay_buffer = ReplayBuffer(state_dim, action_dim, buffer_size)

# Create out agent
agent = DDPGAgent(actor, actor_optimizer, critic, critic_optimizer, tau, 
                  replay_buffer, batch_size, train_start, update_every,
                  env.action_space, epsilon)

In [ ]:
ep_rewards = learn_env(env, agent, gamma, n_steps)
plt.plot(ep_rewards)

Episode: 1, reward: -30.47, time:0.08
Episode: 2, reward: -42.26, time:0.06
Episode: 3, reward: -23.19, time:0.04
Episode: 4, reward: -33.38, time:0.04
Episode: 5, reward: -20.40, time:0.05
Episode: 6, reward: -30.28, time:0.04
Episode: 7, reward: -25.34, time:0.04
Episode: 8, reward: -25.19, time:0.04
Episode: 9, reward: -29.11, time:0.04
Episode: 10, reward: -34.85, time:0.04
Episode: 11, reward: -28.47, time:0.04
Episode: 12, reward: -24.92, time:0.05
Episode: 13, reward: -31.30, time:0.03
Episode: 14, reward: -23.91, time:0.05
Episode: 15, reward: -27.34, time:0.04
Episode: 16, reward: -31.00, time:0.04
Episode: 17, reward: -18.80, time:0.04
Episode: 18, reward: -26.32, time:0.04
Episode: 19, reward: -26.69, time:0.04
Episode: 20, reward: -25.32, time:0.04
Episode: 21, reward: -29.73, time:0.04
Episode: 22, reward: -32.93, time:0.04
Episode: 23, reward: -45.05, time:0.05
Episode: 24, reward: -41.29, time:0.04
Episode: 25, reward: -31.19, time:0.04
Episode: 26, reward: -23.34, time:

In [ ]:
# @markdown ## Run cell to generate video
visualize(env, agent, 200)